In [1]:
import os
import json
import math
import re
import gc  # Import garbage collector module
import logging
from PIL import Image

# Configure logging
logging.basicConfig(
    filename='image_processing.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Configuration Parameters
VERTICAL_THRESHOLD_DEGREES = 9    # 9 degrees

SOURCE_DIR = r"C:\Users\rober\Desktop\GazeDetectionStudy\UnclassifiedDataset"
TARGET_DIR = r"C:\Users\rober\Desktop\GazeDetectionStudy\ClassifiedDataset"

# Ensure the target directory exists
os.makedirs(TARGET_DIR, exist_ok=True)

# Regular expression to extract numbers from filenames
FILENAME_PATTERN = re.compile(r'(\d+)')

# Padding and Shifting Parameters
PADDING_HORIZONTAL = 35  # Horizontal padding in pixels
PADDING_VERTICAL = 20    # Vertical padding in pixels

SHIFT_DOWN_PIXELS = 35   # Shift downward by 30 pixels for "Down"
SHIFT_UP_PIXELS = 15     # Shift upward by 10 pixels for "Up"

def parse_coordinate(coord_str):
    """
    Parses a coordinate string and returns a tuple of floats (x, y).
    Raises ValueError if the coordinate string is invalid.
    """
    coord_str = coord_str.strip('()')
    parts = coord_str.split(',')
    if len(parts) < 2:
        raise ValueError("Incomplete coordinate data.")
    x = float(parts[0].strip())
    y = float(parts[1].strip())
    return x, y

def classify_gaze(phi, vertical_threshold):
    """
    Classifies the gaze direction based on phi angles.
    Returns a classification as "Up", "Center", or "Down".
    """
    abs_phi = abs(phi)

    # Initial Classification based on vertical angle
    if abs_phi >= vertical_threshold:
        return "Up" if phi >= 0 else "Down"
    else:
        return "Center"

def shift_bounding_box(region, min_y, max_y, height):
    """
    Shifts the bounding box based on the region.
    Returns updated (min_y, max_y).
    """
    new_min_y = min_y
    new_max_y = max_y

    if region == "Down":
        shift_amount = SHIFT_DOWN_PIXELS
        potential_new_min_y = min_y + shift_amount
        potential_new_max_y = max_y + shift_amount

        if potential_new_max_y > height:
            # Shift as much as possible without exceeding
            shift_amount = height - max_y
            potential_new_min_y = min_y + shift_amount
            potential_new_max_y = max_y + shift_amount
            logging.warning(f"Adjusting SHIFT_DOWN_PIXELS to {shift_amount} for image to stay within image bounds.")

        new_min_y = potential_new_min_y
        new_max_y = potential_new_max_y

    elif region == "Up":
        shift_amount = SHIFT_UP_PIXELS
        potential_new_min_y = min_y - shift_amount
        potential_new_max_y = max_y - shift_amount

        if potential_new_min_y < 0:
            # Shift as much as possible without going below 0
            shift_amount = min_y  # since min_y - shift_amount = 0
            potential_new_min_y = min_y - shift_amount
            potential_new_max_y = max_y - shift_amount
            logging.warning(f"Adjusting SHIFT_UP_PIXELS to {shift_amount} for image to stay within image bounds.")

        new_min_y = potential_new_min_y
        new_max_y = potential_new_max_y

    return new_min_y, new_max_y

# Iterate over all JSON files in the source directory
for filename in os.listdir(SOURCE_DIR):
    if not filename.endswith('.json'):
        continue  # Skip non-JSON files

    json_path = os.path.join(SOURCE_DIR, filename)
    image_id_match = FILENAME_PATTERN.search(filename)
    if not image_id_match:
        logging.warning(f"Skipping file {filename} due to filename pattern mismatch.")
        continue  # Skip files that don't match the expected pattern

    image_id = image_id_match.group(1)
    image_filename = f"{image_id}.jpg"
    source_image_path = os.path.join(SOURCE_DIR, image_filename)

    # Read and parse the JSON file
    try:
        with open(json_path, 'r') as json_file:
            data = json.load(json_file)
    except json.JSONDecodeError:
        logging.error(f"Skipping file {filename} due to JSON parsing error.")
        continue  # Skip files with invalid JSON

    # Extract the look vector string and parse it
    look_vec_str = data.get('eye_details', {}).get('look_vec', '')
    look_vec_values = look_vec_str.strip('()').split(',')

    # Ensure the look_vec has at least 3 components (x, y, z)
    if len(look_vec_values) < 3:
        logging.error(f"Skipping image {image_id}.jpg due to incomplete look_vec data.")
        continue  # Skip if look_vec is not in the expected format

    try:
        # Parse x, y, z components
        x = float(look_vec_values[0].strip())
        y = float(look_vec_values[1].strip())
        z = float(look_vec_values[2].strip())
    except ValueError:
        logging.error(f"Skipping image {image_id}.jpg due to invalid look_vec values.")
        continue  # Skip if parsing fails

    # Normalize the gaze vector to ensure it's a unit vector
    norm = math.sqrt(x**2 + y**2 + z**2)
    if norm == 0:
        logging.error(f"Skipping image {image_id}.jpg due to zero-length look_vec.")
        continue  # Skip if the vector length is zero

    x_norm = x / norm
    y_norm = y / norm
    z_norm = z / norm

    # Compute phi using the normalized gaze vector (vertical angle)
    phi = math.asin(y_norm)

    # Determine the classification
    region = classify_gaze(
        phi,
        math.radians(VERTICAL_THRESHOLD_DEGREES)
    )

    # Extract eye coordinates from JSON
    interior_margin_2d = data.get('interior_margin_2d', [])
    caruncle_2d = data.get('caruncle_2d', [])
    iris_2d = data.get('iris_2d', [])

    # Collect all the points
    points = []
    for coord_str in interior_margin_2d + caruncle_2d + iris_2d:
        try:
            x_pt, y_pt = parse_coordinate(coord_str)
            points.append((x_pt, y_pt))
        except (ValueError, IndexError):
            continue  # Skip invalid coordinate strings

    if not points:
        logging.error(f"Skipping image {image_id}.jpg due to no valid eye region points.")
        continue  # Skip if no points were found

    # Compute bounding box
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]

    min_x = int(min(x_values))
    max_x = int(max(x_values))
    min_y = int(min(y_values))
    max_y = int(max(y_values))

    # Add separate padding
    min_x = max(min_x - PADDING_HORIZONTAL, 0)
    min_y = max(min_y - PADDING_VERTICAL, 0)

    # Open the image using a context manager
    if not os.path.exists(source_image_path):
        logging.error(f"Image file {source_image_path} does not exist.")
        continue

    try:
        with Image.open(source_image_path) as image:
            width, height = image.size

            # Ensure the coordinates are within the image bounds
            max_x = min(max_x + PADDING_HORIZONTAL, width)
            max_y = min(max_y + PADDING_VERTICAL, height)

            # Apply pixel shifting based on classification
            if region in ["Up", "Down"]:
                # Shift the bounding box
                shifted_min_y, shifted_max_y = shift_bounding_box(
                    region,
                    min_y,
                    max_y,
                    height
                )
                min_y = shifted_min_y
                max_y = shifted_max_y

            # Validate bounding box size
            bbox_width = max_x - min_x
            bbox_height = max_y - min_y

            # Minimum size thresholds
            min_bbox_width = width * 0.05  # 5% of image width
            min_bbox_height = height * 0.05  # 5% of image height

            if bbox_width < min_bbox_width or bbox_height < min_bbox_height:
                logging.warning(f"Skipping image {image_id}.jpg due to small eye region.")
                continue  # Skip if bounding box is too small

            # Crop the image
            cropped_image = image.crop((min_x, min_y, max_x, max_y))

            # Resize the cropped image to 224x224 pixels
            resized_image = cropped_image.resize((224, 224), Image.LANCZOS)

            # Rename and save the resized image
            new_image_filename = f"{image_id}_{region}.jpg"
            target_image_path = os.path.join(TARGET_DIR, new_image_filename)

            try:
                resized_image.save(target_image_path)
                logging.info(f"Cropped, resized, and saved {source_image_path} to {target_image_path}")
            except IOError:
                logging.error(f"Failed to save image {new_image_filename}.")
            finally:
                # Clean up PIL images
                resized_image.close()
                cropped_image.close()

    except IOError:
        logging.error(f"Skipping image {image_id}.jpg due to image opening error.")
        continue  # Skip if the image cannot be opened
    finally:
        # Explicitly delete variables to free memory
        del data
        del points
        del x_values
        del y_values
        gc.collect()  # Force garbage collection
